In [2]:
# Google Driverのマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# ディスクの余分な容量を削除
!pip cache purge
!rm -rf /root/.local/share/Trash/*

Files removed: 0


In [4]:
# ディスク容量、他を確認
!df -h /content

Filesystem      Size  Used Avail Use% Mounted on
overlay         113G   39G   74G  35% /


In [5]:
# HAR-2way-modelのクローンコピー
!git clone https://github.com/JojiOkazaki/HAR-2way-model.git
%cd HAR-2way-model
!ls

Cloning into 'HAR-2way-model'...
remote: Enumerating objects: 237, done.
remote: Counting objects: 100% (237/237), done.
remote: Compressing objects: 100% (133/133), done.
remote: Total 237 (delta 116), reused 210 (delta 92), pack-reused 0 (from 0)
Receiving objects: 100% (237/237), 116.28 KiB | 2.47 MiB/s, done.
Resolving deltas: 100% (116/116), done.
/content/HAR-2way-model
config_base.py	dataset_builder  README.md  training


In [6]:
# .zipの展開をしてデータセットの用意をする
!mkdir -p /content/dataset
!unzip -q "/content/drive/MyDrive/HAR-2way-model/datas/datasets.zip" -d /content/dataset
!df -h /content
!ls -R /content/dataset | head -n 20

Filesystem      Size  Used Avail Use% Mounted on
overlay         113G   98G   15G  87% /
/content/dataset:
datasets

/content/dataset/datasets:
katorilab
ucf101

/content/dataset/datasets/katorilab:
processed_112_32

/content/dataset/datasets/katorilab/processed_112_32:
pt
splits

/content/dataset/datasets/katorilab/processed_112_32/pt:
20250802_094617.pt
20250802_100426.pt
20250802_101041.pt
20250802_101506.pt
20250802_102656.pt


In [7]:
# HAR-2way-model-trainingに必要なライブラリのインストール
!pip install torch==2.5.0 torchvision pyyaml tqdm matplotlib

import torch
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.9 MB/s eta 0:00:00:00:0100:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 17.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 9.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 9.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00:00:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.0 MB/s

In [8]:
%%writefile config_local.py

# config_local.py
from pathlib import Path
RAW_DATA_ROOT = Path("/content/dataset/").resolve()
DATASETS_ROOT = Path(RAW_DATA_ROOT / "datasets").resolve()
DATASET_NAME = "katorilab"
DATASET_ROOT = (DATASETS_ROOT / DATASET_NAME).resolve()
ARTIFACT_ROOT = Path("/content/drive/MyDrive/HAR-2way-model/artifact").resolve()

TRAIN_RATIO = 0.58
VAL_RATIO = 0.14
TEST_RATIO = 0.29
SPLIT_SEED = 42
ENABLE_GROUP_BALANCE = True
GROUP_BALANCE_ALPHA = 5.0

# ファイルパスの存在チェック
print(f"Checking paths...")
print(f"DATASET_ROOT exists: {DATASET_ROOT.exists()} ({DATASET_ROOT})")
# 例: katorilabの中身が見えるか確認
if DATASET_ROOT.exists():
    print(f"Contents of {DATASET_NAME}: {[p.name for p in DATASET_ROOT.iterdir()]}")

assert DATASET_ROOT.exists(), f"Error: {DATASET_ROOT} が見つかりません。解凍パスを確認してください。"

Writing config_local.py


In [15]:
%%writefile training/configs/train.yaml

# train.yaml
logging:
  log_dir: runs
  best_model_name: best_model.pt
  csv_name: loss_history.csv
  graph_dir: graphs
  graph_size: [6, 4]

dataset:
  name: ucf101

data:
  processed_dir: processed_112_32
  split: split01
  # img_pt_dir: processed_112_32/pt
  # skel_pt_dir: processed_112_32/pt
  # train_file_list: processed_112_32/splits/default/train_list.txt
  # val_file_list: processed_112_32/splits/default/val_list.txt
  num_workers: 16

weighted_sampler:
  enabled: true
  alpha: 1.0

model:
  architecture:
    img:
      cnn:
        conv_channels: [3,16,32,64,128,256,256]
        fc_layers: [256]
        dropouts: [0.0,0.2,0.3,0.4,0.5,0.5]
        input_size: [112, 112]
      transformer:
        d_model: 256
        nhead: 8
        num_layers: 3
        dim_ff: 256
        max_len: 32
        dropout: 0.7

    skel:
      stgcn:
        channels: [3,64,64,64,128,128,128,256,256,256]
        dropouts: [0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3]
        temporal_kernel_size: 9

    mlp:
      layers: [512, 512, 768]
      dropouts: [0.4]

  loss_weights: [1.0, 1.0, 1.0]

training:
  batch_size: 16
  epochs: 120
  accum_steps: 4
  max_norm: 5.0 # 勾配が大きすぎないようにする。小さいと学習が遅くなり安定する。
  seed: 42
  patience: 5
  min_delta: 0.005
  recall_k: 1
  
  # proto_ce で学習（(A)）
  loss_mode: proto_ce

  # proto作成用の走査バッチ（VRAM節約で train batch と分けたい場合）
  prototype_batch_size: 64

  # valid人物判定（Trainer側で使う）
  min_valid_t: 16

  # unknown扱いしたい label_id（例: 999 を学習対象から外す）
  unknown_label_ids: [999]

  # proto_ce でも温度は使う（logitsのスケーリング）
  temperature: 0.2

optimizer:
  lr: 0.0001 # 1e-3になると学習できない 0.0005 -> 0.001 (2倍)
  weight_decay: 0.02
  warmup_epochs: 1 # 学習の立ち上がりを遅くすることで序盤を安定させる。小さいと立ち上がりが速くなる。
  min_lr_ratio: 0.1 # 学習後半で学習率を小さくすることで振動を安定させる。小さいと最終学習率が小さくなる。

runtime:
  device: cuda


Overwriting training/configs/train.yaml


In [10]:
!nvidia-smi
print("\n\n")
!cat /proc/cpuinfo | grep "model name" | head -n 1
print("\n\n")
!free -h
print("\n\n")
!cat /etc/issue
print("\n\n")
import sys
print(f"Python version: {sys.version}")

Tue Feb  3 10:03:52 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P0             45W /  400W |       5MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [16]:
!python -m training.main --config training/configs/train.yaml

Checking paths...
DATASET_ROOT exists: True (/content/dataset/datasets/katorilab)
Contents of katorilab: ['processed_112_32']
Create Data Loader...
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
elapsed time: 00:00:00
Build class prototypes from train labels...
elapsed time: 00:01:08
Create Model...
elapsed time: 00:00:00
Training...
  0% 0/477 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/autograd/graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered in